In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls content drive

ls: cannot access 'content': No such file or directory
drive:
'My Drive'  'Shared drives'


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### 데이터 불러오기

In [0]:
youtube_data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/small_total_train_set.csv")
youtube_data

,Unnamed: 0,Unnamed: 0.1,channel_name,subscribers,video_name,days_after_upload,video_duration,hits,likes_num,dislikes_num,category_id,thumbnail
0,0,0,복덩이,104000,"너무나맛있는 '알타리묵은지찜과' ""앞치마에그림"" Altari old paper st...",10,311,1126,40,1,0,https://i.ytimg.com/vi/al52wJgMGyI/hqdefault.j...
1,1,1,복덩이,104000,헌옷으로만든 [앞치마]와 '민들레' 겉절이 'Apron' made with old ...,11,416,1067,85,3,0,https://i.ytimg.com/vi/-xx7shznKec/hqdefault.j...
2,2,2,복덩이,104000,탈모와다이어트에 탁월한고급진 '느타리버섯볶음' 과 프로방스 창문그리기 Stir-fr...,12,265,860,64,0,0,https://i.ytimg.com/vi/m7KC7oEKKuo/hqdefault.j...
3,3,3,복덩이,104000,감기 면연력 간기능보호에 탁월한 '도라지튀김' '민들레튀김' '라벤다그리기' Fri...,13,419,1116,76,1,0,https://i.ytimg.com/vi/FYeuMGEgNWA/hqdefault.j...
4,4,4,복덩이,104000,진달래화전 부침 복덩이표 바삭바삭한 쫀득쫀득비법공개,14,287,622,47,2,0,https://i.ytimg.com/vi/UjLKW2fjkyA/hqdefault.j...
...,...,...,...,...,...,...,...,...,...,...,...,...
13680,31680,31680,키다리형,44000,[VLOG] 체지방 한자릿수 유지식단::뭐든지 과하면 독이다 :: 추억의 VLOG,424,601,47391,554,18,6,https://i.ytimg.com/vi/gPEWgwBoT6I/hqdefault.j...
13681,31681,31681,키다리형,44000,[키용소 2화] 그들의 입맛을 사로잡은 식단 대공개 (맛있게 다이어트하자),426,1146,52216,959,18,6,https://i.ytimg.com/vi/p7Ssy5gB_5A/hqdefault.j...
13682,31682,31682,키다리형,44000,[실속2분팁] 덤벨로우 tip (넓고 두꺼운 등을 위하여),427,142,118107,1300,29,6,https://i.ytimg.com/vi/PI7UN15-5K0/hqdefault.j...
13683,31683,31683,키다리형,44000,[키용소 1화] 살빼는방법을 모르겠어요 :: 위험한 계약,429,1282,78212,1600,42,6,https://i.ytimg.com/vi/iI4-fi64bss/hqdefault.j...


In [0]:
youtube_data = youtube_data[['video_name','category_id']]

In [0]:
print(youtube_data.isnull().values.any())

False


In [0]:
youtube_data['category_id'].value_counts()

5    3582
0    2040
6    2028
2    1897
4    1871
1    1777
3     490
Name: category_id, dtype: int64

### 텍스트 전처리

In [0]:
!pip install soynlp
# 패키지 설치

#### 정규화

In [0]:
from soynlp.normalizer import *
# 정규화 도구 import

In [0]:
for i in youtube_data.index:
  youtube_data['video_name'][i] = emoticon_normalize(youtube_data['video_name'][i],num_repeats=0)
# 반복되는 이모티콘 제거하기
  youtube_data['video_name'][i] = repeat_normalize(youtube_data['video_name'][i],num_repeats=0)
    # 반복되는 단어 길이 제한하기
  youtube_data['video_name'][i] = only_hangle(youtube_data['video_name'][i])
# 한글만 남기고 다 제거하기

In [0]:
youtube_data

In [0]:
titles = list(youtube_data.video_name)
id_num = list(youtube_data.category_id)

# id_num label로 분리하기

#### Stopwords

In [0]:
s_file_name = open('/content/drive/My Drive/Colab Notebooks/ko/stopwords.txt','r',encoding='utf-8')

In [0]:
stop_words = []
for line in s_file_name.readlines():
  stop_words.append(line.rstrip())
s_file_name.close()

In [0]:
print(len(stop_words))

# 불용어 제거하기

#### Tokenize

In [0]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor()
word_extractor.train(titles) # titles = 제목
word_score = word_extractor.extract() # 토크나이저를 위해 통계 학습

In [0]:
from soynlp.tokenizer import MaxScoreTokenizer
word_scores = {word:score.cohesion_forward for word, score in word_score.items()} # 학습된 score 중 cohesion forward 선택
word_tokenizer = MaxScoreTokenizer(scores=word_scores) # word_scores의 통계량을 기준으로 토크나이즈

word_token = []
for sentence in titles:
    temp_X = word_tokenizer.tokenize(sentence, flatten=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stop_words] # 불용어 제거
    word_token.append(temp_X) # 토큰화 된 단어를 문장 단위로 리스트화 [[],[],..[]]

In [0]:
print(len(titles))
print(len(word_token))

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
word_vocab = Tokenizer()
word_vocab.fit_on_texts(word_token)

# 케라스 토크나이저는 띄어쓰기를 기본값으로 토큰화
# 이미 토큰화 된 단어를 띄어쓰기 기준으로 토큰
# 케라스가 index부여, sequence to text, padding등의 편리한 기능을 제거하기 때문에 사용함

In [0]:
print(len(word_vocab.word_index))
print(word_vocab.word_index)

In [0]:
threshold = 3
total_cnt = len(word_vocab.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in word_vocab.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [0]:
vocab_size = total_cnt - rare_cnt # 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거. 0번 패딩 토큰을 고려하여 +1
print('단어 집합의 크기 :',vocab_size)

In [0]:
words_frequency = [w for w,c in word_vocab.word_counts.items() if c < 3]
# 빈도수가 3미만인 단어를 w라고 저장
for w in words_frequency:
    del word_vocab.word_index[w]
    del word_vocab.word_counts[w]

In [0]:
vocab_list = list(word_vocab.word_index.keys())
# 단어 list

In [0]:
print(len(vocab_list))

In [0]:
final_token = []
for sentence in titles:
    temp_X = word_tokenizer.tokenize(sentence, flatten=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stop_words] # 불용어 제거
    temp_X = [word for word in temp_X if word in vocab_list]
    final_token.append(temp_X)
    
# 정규화, 불용어 삭제, 빈도수 2 이하 삭제 된 토큰을 final token으로 [[],[],[]] 이러 ㄴ형식으로 저장

In [0]:
len(final_token)

In [0]:
nan_list = []
for i in range(len(final_token)):
  if len(final_token[i]) == 0:
      nan_list.append(i)

In [0]:
nan_list
# nan값이 있으면 feature extraction할 때 계산 시 (숫자 / 0)으로 infinitie error가 나기 때문에
# nan list를 만들어 차후에 제거해줘야 함

#### 임베딩

In [0]:
from gensim.models import Word2Vec
w2v = Word2Vec(sentences = final_token, size = 100, window = 4, min_count = 1, workers = 4,sample=1e-3, iter=5, sg = 1)

In [0]:
from gensim.models import FastText
fxt = FastText(sentences = final_token, size = 100, window = 4, min_count = 1, workers = 4,sample=1e-3, iter=5, sg = 1)

In [0]:
# 글로브는 w2v과 학습 방식이 다르기 때문에 입력 데이터를 재 작업해줘야 하며
# 뒤에서는 활용하지 못했음
# 토큰화 된 데이터를 다시 글로브에 맞춰서 가공하기 위한 작업
glv_sequences = word_vocab.texts_to_sequences(final_token)
glv_text = word_vocab.sequences_to_texts(glv_sequences)

In [0]:
glove_token = []
for i in range(len(glv_text)):
  glove_token.append(glv_text[i].split())

In [0]:
from glove import Corpus, Glove

corpus = Corpus() 
corpus.fit(glove_token, window=4)
# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성

glv = Glove(no_components=100, learning_rate=0.05)
glv.fit(corpus.matrix, epochs=30, no_threads=3, verbose=False)
glv.add_dictionary(corpus.dictionary)

In [0]:
# 워드 투 벡터의 명령어 옵션이 매우 다양하기 때문에 직접 탐색해봐야 함
# word2vec.wv.vectors 가 우리가 찾고자 하는 벡터들의 값
print(w2v.wv.vectors.shape)
print(fxt.wv.vectors.shape)
print(glv.word_vectors.shape)

In [0]:
len(glv.dictionary.keys()) == len(w2v.wv.index2word)
# 글로브와 w2v가 동일한 단어에 대한 동일한 차원을 가지는 걸 보이기 위함

### 문장의 단어 벡터 평균값으로 특성 구하기

In [0]:
def get_features(words, model, num_features):
    # 출력 벡터 초기화
    feature_vector = np.zeros((num_features), dtype=np.float32)
    
    num_words = 0
    # 어휘 사전 준비
    index2word_set = set(model.wv.index2word)
    
    for w in words:
        if w in index2word_set:
            num_words = 1
            # 사전에 해당하는 단어에 대해 단어 벡터를 더함
            feature_vector = np.add(feature_vector, model[w])
            
    # 문장의 단어 수만큼 나누어 단어 벡터의 평균값을 문장 벡터로 함
    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [0]:
def get_dataset(reviews, model, num_features):
    dataset = list()
    
    for s in reviews:
        dataset.append(get_features(s, model, num_features))
        
    reviewFeatureVecs = np.stack(dataset)
    return reviewFeatureVecs

In [0]:
data_w2v = get_dataset(final_token,w2v,100)
# 전처리 된 데이터를 넣고, 워드 임베딩 벡터와 그 때 압축시킨 차원 값을 넣으면 문장의 특성을 나타내주는 값이 numpy array 형태로 저장됨

In [0]:
data_fxt = get_dataset(final_token,fxt,100)
# 글로브는 함수를 새로 만들어야 해서 하지 못했음

#### 테스트

In [0]:
from sklearn.model_selection import train_test_split

X = data_w2v              # 문장 feature 가 X 변수가 됨
y = np.array(id_num)     # 분리한 label을 array 형태로 바꿈

RANDOM_SEED = 42
TEST_SPLIT = 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, 
                                                    random_state=RANDOM_SEED)

In [0]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)

In [0]:
predicted = lgs.predict(X_test)

print("Accuracy: %f" % lgs.score(X_test, y_test))  #checking the accuracy

In [0]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(kernel='linear', C=1.0)
clf.fit(X_train,y_train)

In [0]:
y_pred = clf.predict(X_test)

In [0]:
accuracy_score(y_test,y_pred)

In [0]:
# 위는 기본 모델에 대한 활용

### 내적으로 특성 구하기

이 부분은 성능이 좋지 않아 활용하지 않음
단어 - 문서 행렬을 만든 뒤 (TF-IDF에서 하듯 문장이 행이되고 단어가 열이 되는 행렬을 만듦)
위에서 구한 word2vec와 같은 임베딩 벡터의 uclid distance를 구한 후 weight를 다시 구함
weight와 단어 - 문서 행렬의 내적을 구하여 문장의 특성을 표현하고자 함

##### DTM만들기

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
cv = CountVectorizer(tokenizer=word_tokenizer,stop_words=stop_words,vocabulary=vocab_list)

In [0]:
tdm = cv.fit_transform(titles).toarray()

In [0]:
print(len(cv.get_feature_names()))

In [0]:
print(tdm.shape)

In [0]:
tdm

##### 가중치 행렬 구하기

In [0]:
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances

distance_matrix = euclidean_distances(w2v.wv.vectors, w2v.wv.vectors)
wght_w2v = np.exp(-(distance_matrix ** 2) / (2 * np.var(distance_matrix)))

In [0]:
distance_matrix = euclidean_distances(fxt.wv.vectors,fxt.wv.vectors)
wght_fxt = np.exp(-(distance_matrix ** 2) / (2 * np.var(distance_matrix)))

In [0]:
distance_matrix = euclidean_distances(glv.word_vectors, glv.word_vectors)
wght_glv = np.exp(-(distance_matrix ** 2) / (2 * np.var(distance_matrix)))

In [0]:
print(wght_w2v.shape)
print(wght_fxt.shape)
print(wght_glv.shape)

##### 내적

In [0]:
feat_w2v = np.dot(tdm,wght_w2v)

In [0]:
feat_fxt = np.dot(tdm,wght_fxt)

In [0]:
feat_glv = np.dot(tdm,wght_glv)

In [0]:
print(feat_w2v.shape)
print(feat_fxt.shape)
print(feat_glv.shape)

In [0]:
def slower_dot (A, B):
    """Low-memory implementation of dot product"""
    #Assuming A and B are of the right type and size
    R = np.empty([A.shape[0], B.shape[1]])
    for i in range(A.shape[0]):
        for j in range(B.shape[1]):
            R[i,j] = np.dot(A[i,:], B[:,j])
    return R

In [0]:
#R[0,1] = np.array(np.dot(tdm_32[1,:],weight_matrix_w2v[:,1]))

In [0]:
#dotproduct_test = slower_dot(tdm_test_32,test_wght_w2v)

In [0]:
#dotproduct_test.shape

In [0]:
dot_w2v = np.dot(tdm,w2v.wv.vectors)

In [0]:
dot_fxt = np.dot(tdm,fxt.wv.vectors)

In [0]:
dot_glv = np.dot(tdm,glv.word_vectors)

In [0]:
print(dot_w2v.shape)
print(dot_fxt.shape)
print(dot_glv.shape)

In [0]:
for i in range(len(dot_w2v)):
  dot_w2v[i] = dot_w2v[i]/len(dot_w2v[i])

In [0]:
dot_w2v.shape

##### 테스트

In [0]:
from sklearn.model_selection import train_test_split

X = feat_w2v
y = np.array(id_num)

RANDOM_SEED = 42
TEST_SPLIT = 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, 
                                                    random_state=RANDOM_SEED)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB # 다항분포 나이브 베이즈 모델
from sklearn.metrics import accuracy_score #정확도 계산

In [0]:
mod = MultinomialNB()
mod.fit(X_train,y_train)

In [0]:
predicted = mod.predict(X_test) #테스트 데이터에 대한 예측

In [0]:
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

In [0]:
from sklearn.model_selection import train_test_split

X = dot_w2v
y = np.array(id_num)

RANDOM_SEED = 42
TEST_SPLIT = 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, 
                                                    random_state=RANDOM_SEED)

In [0]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)

In [0]:
predicted = lgs.predict(X_test)

print("Accuracy: %f" % lgs.score(X_test, y_test))  #checking the accuracy

In [0]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(kernel='linear', C=1.0)
clf.fit(X_train,y_train)

In [0]:
y_pred = clf.predict(X_test)

In [0]:
accuracy_score(y_test,y_pred)